# Análisis Exploratorio de Datos

## Introducción

Uno de los primeros pasos para desarrollar un proyecto de ciencia de datos es entender los datos. Para esto, una técnica que se utiliza un **Análisis Exploratorio de Datos**, en el que se exploran los datos y se resumen sus principales características. Esta técnica es clave en la comprensión de los datos. El principal objetivo es adquirir intruición sobre el contenido de los datos, para ayudar a enmarcar las preguntas que se harán y para proponer posibles caminos para llegar a respuestas a esas preguntas.

En esa sección trabajaremos con un conjunto de datos estándar muy usado en aprendizaje automático, que consiste en informaciones sobre viviendas en districtos del estado de California, EEUU. En realidad usaremos una versión ligeramente modificada, preparada por Aurélien Géron.

**Plan**

La idea para hoy es avanzar en los primeros pasos de un proyecto de ciencia de datos: la **comprensión del problema** y la **comprensión de los datos**.

Como vimos, ambas tareas se realizan en paralelo, yendo y viniendo de un lado al otro.

En este notebook vamos a:

1. Recolectar los datos.

2. Descubrir y visualizar los datos de forma a aquirir intuición sobre ellos.
    * ¿Qué tipos de preguntas podemos intentar responder con este conjunto de datos?

    * ¿Necesitamos de más datos?

3. Definir el proyecto.

La semana que viene, vamos a dar un paso hacia la **preparación de los datos**.

## Celdas preparatorias

Primero necesitaremos ejecutar algunas celdas de código para establecer el entorno. 

**Nota importante**: algunas de las cosas que vienen abajo están basadas en el libro de A. Géron y su repositorio git asociado [repositorio GitHub](https://github.com/ageron/handson-ml2).

In [ ]:
# se requiere Python versión ≥3.5
import sys
assert sys.version_info >= (3, 5)

# se requiere Scikit-Learn ≥0.20  
import sklearn
assert sklearn.__version__ >= "0.20"

# imports frecuentes
import os
import numpy as np

# Para hacer figuras bellas
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Crea un directorio "imagenes", si no existe
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "01_AED"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "imagenes", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

# Una función práctica para guardar imágenes
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignorar avisos sin utilidad (ver SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Recolectando los datos

El primer paso es acceder a los datos. Un archivo del tipo `csv` (valores separados por coma) está disponible en un repositorio público y también se los mandamos por Discord. Esta celda bajará primero el archivo que se necesita.

In [ ]:
import tarfile

HOUSING_PATH = "datasets"
DOWNLOAD_ROOT = "https://github.com/IAI-UNSAM/datasets/raw/master/"
HOUSING_URL = DOWNLOAD_ROOT + "housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    !wget {housing_url} -P {housing_path}
    tgz_path = os.path.join(housing_path, "housing.tgz")
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    return


In [ ]:
# Ejecuta la función que acabamos de crear
fetch_housing_data()

Ahora, importemos el módulo  `pandas` y definamos una función para leer los datos en una de las clases de `pandas`, un `DataFrame`.

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing_mod.csv")
    return pd.read_csv(csv_path)

# La función carga los datos como una instancia DataFrame de Pandas
housing = load_housing_data()

## Exploración de los datos

Ahora tenemos los datos en un formato muy práctico (y muy usado), una instancia `pandas.DataFrame`, referida por la variable `housing`.

Acá mostraremos algunas formas bastante básicas de mirar los contenidos de una tabla. Recordar siempre que se puede agregar `?` a un objeto o método para acceder a su documentación (por ejemplo, `housing?` or `housing.head?`).

***

**1.** Para ver las primeras líneas de una tabla, use el método  `head`, que permite definir un argumento opcional indicando cuantas líneas mostrar.

In [ ]:
housing.head()

Este método acepta un argumento opcional, `n`, que define cuántas líneas se muestras. 

**Pruébenlo**. Remplacen los puntos `...` por su código. Reucerden que siempre pueden usar `?` al final del método para leer la documentación.

In [ ]:
housing.head(...)

Por otro lado, el nombre de las columnas se puede ver con el atributo `columns`.

In [ ]:
print(housing.columns)

**2.** El método `.info` provee información básica sobre el contenido de cada columna.

In [ ]:
housing.info()

Vemos que hay **20640** *entradas*, con 10 columnas cada. Cada una de esas columnas representa un *atributo* de las entradas. 

**Pregunta**: ¿Sabemos a qué hace referencia cada fila? Es decir, ¿sabemos cuáles son las unidades de este dataset?



Excepto `ocean_proximity`, todos los atributos pueden describirse con variables numéricas (`float64`). Además, faltan datos en la columna `total_bedrooms`. Tendremos que lidiar con eso después...

Démosle una mirada rápida a la columna `ocean_proximity`. Que valores toma? (**Nota**: podemos acceder a las columnas como atributos *data frame*, o con una sintaxis tipo diccionario. En otras palabras, los dos comandos en la celda de abajo son equivalentes).

In [ ]:
print(housing['ocean_proximity'])
print(housing.ocean_proximity)

Para ver los distintos valores que esta columna toma, usemos el método `unique`.

In [ ]:
print(housing['ocean_proximity'].unique())
print(housing.ocean_proximity.unique())

Podemos ir un paso más adelante y ver cuántas veces aparece cada valor.

Para esto, usamos el método `value_counts`.

In [ ]:
print(housing.ocean_proximity.value_counts())

Podemos acá colar nuestro primer gráfico, porque es muy simple hacerlo en este caso.

In [ ]:
# Guarda los resultados en un nuevo dataframe
distritos_por_clase = housing.ocean_proximity.value_counts()

# Hace un plot
distritos_por_clase.plot(kind='bar', 
                         xlabel='Cercanía al océano',
                         ylabel='Cantidad de distritos')#, figsize=(8, 6))

In [ ]:
# O una versión normalizada al total de entradas
print(housing.ocean_proximity.value_counts(normalize=True))

**Pregunta**: están todas las clases igualmente representadas?

A esto uno se refiere como "desbalance de classes" y será muy importante para los algoritmos de clasificación.

***

**3.** Para columnas numéricas, podemos obtener facilmente estadísticas descriptivas simples.

El método `describe` calcula el promedio, la desviación estándar y otras medidas estadística que describem los datos. 



In [ ]:
housing.describe()


**Pregunta**: ¿ven algo sorprendente o que valga la pena mencionar? De nuevo, ¿se entiende a que se refiere cada línea en el conjunto de datos (no en esta tabla)?

 **Nota 1**: esto excluye automaticamente `NaN` de los cálculos; vease la entrada `count` en `total_bedrooms`.
 
 **Nota 2**: se puede definir qué percentiles se calculan con el argumento `percentiles` del método `describe`.

**Ejercicio**. Miren la documentación del método usando `?`. ¿Cómo cambia la salida de este método si usamos el argumento `include='all'`.

**¡Prueben!** Intenten entender lo que pasa.

In [ ]:
housing.describe(include=...)

***
**4.** Podemos ordenar la tabla para ver cuales son los districtos donde algunos valores adquieren valores extremos.

Para eso usamos el método `sort_values` y definimos la columna según la cual queremos ordenar la tabla. En el ejemplo de abajo ordenamos la pabla por la columna `median_income`.

In [ ]:
housing.sort_values(by='median_income', ascending=False)

**Pregunta**: presten atención a los valores de las primeras y las últimas filas de las variables `median_income` y `median_house value`. ¿Qué identifican? ¿Pueden explicar qué puede estar pasando?



**Nota**: el mismo método se puede ussar para ordenar por más de una columna pasando una lista por el argunento `by`. Por ejemplo, 


In [ ]:
housing.sort_values(by=['median_income', 'total_rooms'], ascending=False)

ordenará la tabla por ingreso (`median_income`) y después por el número total de ambientes (`total_rooms`).

**Ejercicio**. Verifiquen que está dando lo que esperamos, y prueben con otro par de variables. ¿Qué creen que hace el argumento `ascending`?

In [ ]:
housing.sort_values(by=[..., ...], ascending=False)

***

**5.** Podemos agrupar entradas según el valor de una dada variable. Eso es útil para un análisis rápido de los datos.

Para hacer eso, usamos otro método de la clase `DataFrame` llamado `groupby`. 


In [ ]:
grouped_housing = housing.groupby(by='ocean_proximity')


Esto devuelve un nuevo objeto, sobre el que podemos calcular distintas métricas. 

**Ejercicio**. Usen el método `mean()` en el DataFrame `housing` y en el nuevo objeto `grouped_housing`. Discutan la diferencia en el resultado.

In [ ]:
housing.mean()

In [ ]:
grouped_housing.mean()

**Pregunta**: ya podemos decir algo basados en esa tabla? Si pueden, ¡**felicitaciones** por su primera intuición obtenida partir de los datos! ¿Qué aprendieron sobre estos districtos? 

De forma similar al método `sort_values`, se puede pasar una lista al argumento `by` para crear una clasificación anidada.

**Ejercicio**. Discutan qué hace la próxima celda.

In [ ]:
housing.groupby(by=['ocean_proximity', 'total_bedrooms']).count()

***

**6.** Por último, pero no menos importante, podemos explorar estadísticas que involucran más de una variable. La más común es el coeficiente de correlación de Pearson.

Una definición formal está fuera del ámbito del curso, pero digamos solamente que el coeficiente de Pearson de dos variables $X$ y $Y$, escrito como $\hat{\rho_{XY}}$, es un estimador del _verdadero_ coeficiente de correlación y se calcula como:

<!-- $$
\hat{\rho_{XY}} = r = \frac{\hat{\mathrm{cov}}_{XY}}{\hat{\sigma}_X \hat{\sigma}_Y}\;\;,
$$
donde 

$$
\hat{\sigma}_X^2 = \frac{1}{N - 1}\sum_{i=1}^N (x_i - \bar{X})^2\;\;,
$$
$$
\hat{\mathrm{cov}}_{XY} = \frac{1}{N - 1}\sum_{i=1}^N (x_i - \bar{X})(y_i - \bar{Y})\;\;,
$$
y
$$
\hat{\mu_X} = \bar{X}\;\;.
$$ -->

<!-- Notar el significado de esas ecuaciones.  -->

El coeficiente de Pearson aumenta cuando ambas variables se alejan de sus valores promedios en la misma dirección y disminuye si se mueven en direcciones opuestas. Sus valores extremos son -1 y 1, para una (anti-)correlation perfecta entre ambas variables. 

Luego cero significa que las variables no están correlacionadas, correcto? .... **No!**

Para ganar un poco de intuición, discutiamos esta figura, obtenida de [Wikipedia](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient):

<center><img src="https://upload.wikimedia.org/wikipedia/commons/d/d4/Correlation_examples2.svg"></center>

Los números sobre cada sub-gráfico indican el valor del coeficiente de Pearson. 

**Pregunta:** Estudien con detenimiento cada una de las filas de este gráfico. ¿Qué pueden concluir?

El coeficiente de Pearson entre todos los pares de variables es calculado facilmente con `pandas` usando el método `corr`.

**Tip**: los argumentos del método permiten elegir la técnica usada para calcular el coeficiente de correlación. Por defecto se calcula el coeficiente de Pearson descripto más arriba.

In [ ]:
corr_matrix = housing.corr()

In [ ]:
# Qué tamaño espera que tenga `corr_matrix`?
print(corr_matrix.shape)

Podemos mandar imprimir la matriz, pero es más interesante hacer un gráfico de ella.

In [ ]:
print(corr_matrix)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(corr_matrix)

# Definir las marcaciones 
xt = plt.xticks(np.arange(9), housing.columns[:-1], rotation=45, ha='right', va='top')
yt = plt.yticks(np.arange(9), housing.columns[:-1], rotation=0, ha='right', va='center')

# Definir la barra de colores
plt.colorbar(label='Pearson CC')

**Ejercicio**.

* Miren con detenimiento esta figura.

* Mencionen dos pares de variables que estén correlacionadas (positivamente; valores amarillos) y un par que estén claramente anti-correlacionadas (correlacionadas negativamente; valores violetas). Indiquen además al menos dos pares de variables que estén muy débilmente correlacionads (colores verdes).

* Noten la estructura en bloques del gráfico de arriba. ¿Pueden explicarlo?

* ¿Qué atributo correlaciona más fuertemente con la mediana del valor de las casas? Si tuviéramos que elegir un único atributo para usar como predictor del valor de las casas ¿cuál sería? ¿Por qué?



## Preguntas

Aquí no hay respuestas correctas o equivocadas

1) Qué preguntas pueden ser contestadas con esos datos?

2) Cuales de esas preguntas pueden ser atacadas usando aprendizaje automático? En esos casos, cual es el mapeo de A a B que necesitaría que el algoritmo aprenda?

3) Qué variables piensa que serán más relevantes en ese caso?

## Visualización

Una forma excelente de ganar intuición sobre un conjunto de datos es hacer gráficos.

El paquete más usual para hacer gráficos en python es el `matplotlib`. En particular, muchas de las funciones que usaremos están representadas en el sub-paquete `pyplot` (importado arriba como `plt`).

In [ ]:
plt.plot?

`Pandas` tiene un buen empaquetamiento para `matplotlib.pyplot`, pero intentaremos usar exclusivamente las funciones de `pyplot`.

Otro paquete muy bueno con una interacción fuerte con `pandas` es `seaborn`, pero para evitar la proliferación de paquetes, no discutiremos ese en este curso.

### Histograma

In [ ]:
#h = plt.hist(housing.median_income, 50)
housing.median_income.plot(kind='hist', 
                           bins=100, 
                           )

plt.gca().set_xlabel('Mediana del ingreso')

### Gráficos de violin

Los gráficos de violin son una buena forma de descubrir compartamientos en distribuciones de datos, separados por categorías. El gráfico de violin consiste en un gráfico vertical de la distribución de una variable usando una especie de filtro de suavización (en realidad una técnica de estimación de densidad).

Ejemplo. Ya vimos cómo algunos parámetros cambian con la categoría `ocean_proximity`. Intentemos ver ese comportamiento con un gráfico simple.

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111)

categories = housing.ocean_proximity.unique()

# Preparar los datos para que sean graficados. Separar por categoría en ocean_proximity y transformar el array
datos = housing.groupby('ocean_proximity').median_house_value.apply(np.array)

ax.boxplot(datos, showmeans=True)
#ax.violinplot(datos, showextrema=True, showmeans=True, points=1000)

# Definir marcadores para los nombres de las clases
# ax.xaxis.set_ticks(range(1, len(categories)+1))
ax.xaxis.set_ticklabels(datos.index)

# Definir el nombre del eje y
ax.set_ylabel('Mediana del valor de las propiedades', fontsize=16)

Noten que es mucho más fácil visualizar la diferencia entre las distintas clases que mirando la tabla arriba.

¡Los gráficos son muy importantes para comunicar los resultados!

### Gráficos de correlación (*corner plots*)

Discutimos el coeficiente de Pearson para entender la correlación (lineal) entre caractarísticas. También podemos usar una herramienta de visualización para ver como pares de características están relacionados.

Una forma simple de hacer eso es usando la función `scatter_matrix` de `pandas.plotting`.

Puede llevar bastante tiempo hacer una gráfica de todos los pares de variables si mantenemos todas las líneas. Por lo tanto, vamos a hacer el conjunto de datos un poco más *delgado*. Para eso usaremos el método `iloc`.

In [ ]:
axes = pd.plotting.scatter_matrix(housing.iloc[::20], figsize=(12, 12), alpha=0.5, hist_kwds={'bins': 25})

# Roto las etiquetas (puramente estético)
for ax in axes.flatten():
    ax.xaxis.label.set_rotation(45)
    ax.xaxis.label.set_ha('right')
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha('right')

**Pregunta**: ¿ven algo interesante que no ha sido revelado por la matriz de correlación de Pearson?

**Pregunta 2**: ¿entienden qué representan los histogramas en la diagonal? Si eso no está claro, intenten experimentar con la función `plt.hist` y un único atributo (vea la celda abajo)

In [ ]:
_ = plt.hist(housing.median_house_value, bins=25, histtype='step')
plt.xlabel('Median House Value')

### Gráficos de Dispersión (*Scatter Plots*)

Hagamos un gráfico simple de dispersión de Latitud vs. Longitud

**Tip**. Note que uno no necesita crear las instancias figura (`Figure`) y ejes (`Axes`) como arriba. Podemos simplemente usar la función el el paquete `plt`.

In [ ]:
# Hagamos un gráfico simple de latitud vs longitud
scatter = plt.scatter(housing.longitude, housing.latitude)
plt.xlabel('Longitud')
plt.ylabel('Latitud')

Esta es una gráfica bastante fea. No nos dice mucho. Podemos ver la forma de California, pero nada más.

Seguro que podemos conseguir algo mejor.

Para empezar, hagamos el mismo gráfico que arriba usando transparencia para los puntos.

(Para controlar el tamaño de la figura, creamos una instancia `figure` primero)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
scat = ax.scatter(housing.longitude, housing.latitude, alpha=0.2)

ax.set_xlabel('Longitud [grados]', fontsize=16)
ax.set_ylabel('Latitud [grados]', fontsize=16)
# save_fig("better_visualization_plot")

**Ejercicio**

***
* Incluyan más variables en un gráfico de *scatter* usando los argumentos de color (`color` o `c`) y tamaño (`size` o `s`). Además, recuerde que puede mejorar algunos gráficos haciendo los puntos semi-transparantes con `alpha`.

**Nota**: pueden salvar las figuras que quieran con la función `save_fig` definida arriba.

**Ahora** podemos ver que algunas regiones son más densamente pobladas que otras.

*** 

**Pregunta**: qué otras variables incluiría? Complete el modelo abajo para crear nuevos gráficos de dispersión.

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)

color_feature = 'total_bedrooms'
size_feature = 'total_rooms'

scat = plt.scatter(housing.longitude, housing.latitude, alpha=0.4, 
                  c=housing[color_feature], cmap='jet',
                  s=housing[size_feature]/300, label=size_feature)
plt.colorbar(scat, label=color_feature)

ax.set_xlabel('Longitud [grados]', fontsize=16)
ax.set_ylabel('Latitud [grados]', fontsize=16)

leg = ax.legend()

**Tip**. Quizás quiera usar un factor numérico para definir la escala de las características de tamaño y color, de forma a usar correctamente el rango completo de tamaños y colores.

**Tip 2**. También puede querer experimentar con diferentes escalas de colores, cambiando el argumento `cmap` (de *color map*). Una lista de los mapas de colores se puede obtener ejecutando la celda arriba.

**Tip 3**. Copie el código arriba en una nueva celda para hacer otras pruebas.

Por fin, para el *golpe de gracia*, puede superponer su gráfica favorita sobre un lindo mapa de California. El código abajo es tomado del repositório de A. Géron.

In [ ]:
# Baja la imagen de California
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)
DOWNLOAD_ROOT = "https://github.com/IAI-UNSAM/datasets/raw/master/"
filename = "california.png"
print("Downloading {} from {}".format(filename, DOWNLOAD_ROOT))

MAP_URL = DOWNLOAD_ROOT + "images/" + filename

def fetch_california_map(map_url=MAP_URL, image_path=IMAGES_PATH):
    os.makedirs(image_path, exist_ok=True)
    !wget {map_url} -P {image_path}
    return

fetch_california_map()

In [ ]:
import matplotlib.image as mpimg

# Baja la imagen de California
DOWNLOAD_ROOT = "https://github.com/IAI-UNSAM/datasets/raw/master/"
filename = "california.png"
print("Downloading {} from {}".format(filename, DOWNLOAD_ROOT))

MAP_URL = DOWNLOAD_ROOT + "images/" + filename

# Función para bajar un map de California
def fetch_california_map(map_url=MAP_URL, image_path=IMAGES_PATH):
    os.makedirs(image_path, exist_ok=True)
    !wget {map_url} -P {image_path}
    return

fetch_california_map()

# Lee la imagen de California
california_img = mpimg.imread(os.path.join(IMAGES_PATH, filename))

In [ ]:

# Con esa imagen, prepara un gráfico
fig = plt.figure(figsize=(12, 9))

color_feature = 'median_income'
size_feature = 'population'

scat = plt.scatter(housing.longitude, housing.latitude, alpha=0.4, 
                  c=housing[color_feature], cmap='jet',
                  s=housing[size_feature]/1000, label=size_feature)

# Muestra la imagen de abajo
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5,
           cmap=plt.get_cmap("jet"))

# Nombre de los ejes
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

# Adecuar la barra de colores
colorfeat = housing[color_feature]
tick_values = np.linspace(colorfeat.min(), colorfeat.max(), 11)
cbar = plt.colorbar(ticks=tick_values/colorfeat.max())
# cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label(color_feature.replace('_', ' ').title(), fontsize=16)

plt.legend(fontsize=16)
# save_fig("california_housing_prices_plot")
plt.show()

**Estas** gráficas finales contienen mucha información.

**Pregunta.** Podemos relacionar esa gráfica a nuestro conocimiento sobre el mercado habitacional?

## Conclusión

Fuimos desde un archivo separado por comas (`.csv`) hasta visualizaciones muy informativas, explorando como las variables dependen entre ellas y también algunas peculiaridades en algunas columnas (como valores que faltan). Todavía no hemos usado ningún concepto estadístico (bueno, quizás el coeficiente de Pearson cuente como uno), ni tampoco un modelo matemático.

Este tipo de anális exploratório de datos es requerido como un primer paso en todos los proyectos de aprendizaje automático y es una herramienta poderosa para maximizar lo que se puede extraer de los datos.

In [ ]:
pd.get_dummies(housing.ocean_proximity)

In [ ]:
pd.concat([housing, pd.get_dummies(housing.ocean_proximity)], axis=1).drop(['ocean_proximity'], axis=1)

## Extras

### Codificación *one-hot* usando `pandas`

In [ ]:
pd.get_dummies(housing.ocean_proximity)

Ahora concatene nuevas columnas con uma tabla anterior y remueva la variable antigua `ocean_proximity`.

In [ ]:
new_housing = pd.concat([housing, pd.get_dummies(housing.ocean_proximity)], axis=1).drop(['ocean_proximity'], axis=1)
new_housing

### Codificación por color

In [ ]:
# Generar códigos de colores
from sklearn import preprocessing as pre
ordenc = pre.OrdinalEncoder()
color_code = ordenc.fit_transform(housing.loc[:, ['ocean_proximity',]]).flatten()

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)

# Usar códigos de color como color
size_feature = 'total_rooms'
color_feature = 'Ocean Proximity'

scat = plt.scatter(housing.longitude, housing.latitude, alpha=0.2, 
                  c=color_code, cmap='jet',
                  s=housing[size_feature]/500, label=size_feature)
plt.colorbar(scat, label=color_feature)
leg = ax.legend()


# **Ejercicios**

Se observa una correlacion de pearson de 0.7 entre las variables Median House Value y Median Income.
A que se puede deber esta correlación?

Grafique ambas variables por separado, como son sus distribuciones?
(Incluir en su respuesta términos como media, skewness, simetría, valores atípicos)
Importante: que significan los valores que toman estas variables? Son pesos, dolares, criptomonedas?

Grafique Median House value en función de Median Income y describa la relación entre estas variables a partir del gráfico.